In [110]:
using Pkg
Pkg.activate("./")
using Suppressor
@suppress begin
    using Oceananigans, CairoMakie, TimestepperTestCases, Statistics
end

  Activating project at `~/development/TimestepperTestCases.jl`


In [ ]:
folder = "idealized_coast_unforced/"
closure = "unforced"

rl = @suppress TimestepperTestCases.load_idealized_coast(folder, closure, "split_free_surface_lowres_",    "SplitRungeKutta3");
al = @suppress TimestepperTestCases.load_idealized_coast(folder, closure, "split_free_surface_lowres_",    "QuasiAdamsBashforth2");
ri = @suppress TimestepperTestCases.load_idealized_coast(folder, closure, "implicit_free_surface_lowres_", "SplitRungeKutta3");

In [ ]:
function line_mean(field)
  Ny, Nz = size(field)
  lm = zeros(Nz)
  sz = zeros(Int, Nz)
  @inbounds for k in 1:Nz, j in 1:Ny
        if !isnan(field[j, k])
            lm[k] += field[j, k]
            sz[k] += 1
        end
   end
   return lm ./ sz
end 

using LaTeXStrings

ticks(vec) = (vec, latexstring.(string.(vec)))

function running_mean(v, points)
    n  = length(v)
    rm = zeros(length(v) - 2points+1)
    for i in points+1:n-points
        rm[i-points] = mean(v[i - points:i+points])
    end
    return rm[1:end-1]
end

In [ ]:
c1 = :deepskyblue 
c2 = :blue
c3 = :orange1 
c4 = :steelblue

mvv = 8

zC  = ri[:Abx].grid.z.cᵃᵃᶜ[2:39] 
fig = Figure(size = (750, 350))

xticks1 = ([0, 1e-5, 2e-5], [L"0", L"10^{-5}", L"2 \cdot 10^{-5}"])
xticks2 = ([0, 3e-5, 6e-5, 9e-5], [L"0", L"3\cdot 10^{-5}", L"6 \cdot 10^{-5}", L"9 \cdot 10^{-5}"])
xticks3 = ([0, 2e-5, 4e-5, 6e-5], [L"0", L"2\cdot 10^{-5}", L"4 \cdot 10^{-5}", L"6 \cdot 10^{-5}"])
xticks4 = ([0, 5e-5, 1e-4, 1.5e-4], [L"0", L"5\cdot 10^{-5}", L"10^{-4}", L"1.5 \cdot 10^{-4}"])

xlabel = L"\kappa_{num} \text{ [m}^2\text{s}^{-1}\text{]}"

ax1 = Axis(fig[1, 1]; 
           xlabel,
           xticks = ([0, 1e-5, 2e-5], [L"0", L"10^{-5}", L"2\cdot 10^{-5}"]),
           ylabel = L"\text{Depth [m]}",
           yticks = ticks([-100, -50, 0]),
           title=L"t = \text{%$(Oceananigans.Utils.prettytime(rl[:Abz].times[120]))}") 
ax2 = Axis(fig[1, 2]; 
           xlabel,
           xticks = ([0, 2e-6, 4e-6], [L"0", L"2\cdot 10^{-6}", L"4\cdot 10^{-6}"]),
           ylabel = "",
           yticks = ([-100, -50, 0], ["", "", ""]),
           title=L"t = \text{%$(Oceananigans.Utils.prettytime(rl[:Abz].times[240]))}") 
ax3 = Axis(fig[1, 3]; 
           xlabel,
           xticks = ([0, 1e-6, 2e-6], [L"0", L"10^{-6}", L"2\cdot 10^{-6}"]),
           ylabel = "",
           yticks = ([-100, -50, 0], ["", "", ""]),
           title=L"t = \text{%$(Oceananigans.Utils.prettytime(rl[:Abz].times[360]))}") 
ax4 = Axis(fig[1, 4]; 
           xlabel,
           xticks = ([0, 5e-7, 1e-6], [L"0", L"5\cdot 10^{-7}", L"10^{-6}"]),
           ylabel = "",
           yticks = ([-100, -50, 0], ["", "", ""]),
           title=L"t = \text{%$(Oceananigans.Utils.prettytime(rl[:Abz].times[480]))}") 

α   = 1.0
ϵ   = 1e-20
it  = 120
κrl = abs.(line_mean(interior(Field(mean(rl[:Abz][it] + rl[:Aby][it] + rl[:Abx][it], dims=1) / (mean(rl[:Gbz][it], dims=1) + ϵ) / 2), 1, :, :)))[2:39]
κri = abs.(line_mean(interior(Field(mean(ri[:Abz][it] + ri[:Aby][it] + ri[:Abx][it], dims=1) / (mean(ri[:Gbz][it], dims=1) + ϵ) / 2), 1, :, :)))[2:39]
κal = abs.(line_mean(interior(Field(mean(al[:Abz][it] + al[:Aby][it] + al[:Abx][it], dims=1) / (mean(al[:Gbz][it], dims=1) + ϵ) / 2), 1, :, :)))[2:39]
lines!(ax1, κrl, zC, color = (c1, α))
lines!(ax1, κri, zC, color = (c2, α))
lines!(ax1, κal, zC, color = (c3, α))
it  = 240
κrl = abs.(line_mean(interior(Field(mean(rl[:Abz][it] + rl[:Aby][it] + rl[:Abx][it], dims=1) / (mean(rl[:Gbz][it], dims=1) + ϵ) / 2), 1, :, :)))[2:39]
κri = abs.(line_mean(interior(Field(mean(ri[:Abz][it] + ri[:Aby][it] + ri[:Abx][it], dims=1) / (mean(ri[:Gbz][it], dims=1) + ϵ) / 2), 1, :, :)))[2:39]
κal = abs.(line_mean(interior(Field(mean(al[:Abz][it] + al[:Aby][it] + al[:Abx][it], dims=1) / (mean(al[:Gbz][it], dims=1) + ϵ) / 2), 1, :, :)))[2:39]
lines!(ax2, κrl, zC, color = (c1, α))
lines!(ax2, κri, zC, color = (c2, α))
lines!(ax2, κal, zC, color = (c3, α))
it  = 360
κrl = abs.(line_mean(interior(Field(mean(rl[:Abz][it] + rl[:Aby][it] + rl[:Abx][it], dims=1) / (mean(rl[:Gbz][it], dims=1) + ϵ) / 2), 1, :, :)))[2:39]
κri = abs.(line_mean(interior(Field(mean(ri[:Abz][it] + ri[:Aby][it] + ri[:Abx][it], dims=1) / (mean(ri[:Gbz][it], dims=1) + ϵ) / 2), 1, :, :)))[2:39]
κal = abs.(line_mean(interior(Field(mean(al[:Abz][it] + al[:Aby][it] + al[:Abx][it], dims=1) / (mean(al[:Gbz][it], dims=1) + ϵ) / 2), 1, :, :)))[2:39]
lines!(ax3, κrl, zC, color = (c1, α))
lines!(ax3, κri, zC, color = (c2, α))
lines!(ax3, κal, zC, color = (c3, α))
it  = 480
κrl = abs.(line_mean(interior(Field(mean(rl[:Abz][it] + rl[:Aby][it] + rl[:Abx][it], dims=1) / (mean(rl[:Gbz][it], dims=1) + ϵ) / 2), 1, :, :)))[2:39]
κri = abs.(line_mean(interior(Field(mean(ri[:Abz][it] + ri[:Aby][it] + ri[:Abx][it], dims=1) / (mean(ri[:Gbz][it], dims=1) + ϵ) / 2), 1, :, :)))[2:39]
κal = abs.(line_mean(interior(Field(mean(al[:Abz][it] + al[:Aby][it] + al[:Abx][it], dims=1) / (mean(al[:Gbz][it], dims=1) + ϵ) / 2), 1, :, :)))[2:39]
lines!(ax4, κrl, zC, color = (c1, α), label=L"\text{RK3-SE}")
lines!(ax4, κri, zC, color = (c2, α), label=L"\text{RK3-IM}")
lines!(ax4, κal, zC, color = (c3, α), label=L"\text{QAB2-SE}")
ylims!(ax1, -100, 0)
ylims!(ax2, -100, 0)
ylims!(ax3, -100, 0)
ylims!(ax4, -100, 0)
Legend(fig[1, 5], ax4)
display(current_figure())
path = "/Users/simonesilvestri/Dropbox (MIT)/Apps/Overleaf/timestepper/"

CairoMakie.save(path * "dissipation-coast.pdf", fig)

In [ ]:
fig = Figure(size = (900, 250))
jr  = 1:60
kr  = 20:40
zC  = ri[:S].grid.z.cᵃᵃᶜ[kr]
yC  = ri[:S].grid.yᵃᶜᵃ[jr] ./ 1e3
ax1  = Axis(fig[1, 1], title = L"\text{QAB2-SE - RK3-SE}",
                       ylabel = L"\text{Depth [m]}",
                       xlabel = L"\text{y [km]}",
                       xticks = ticks([0, 25, 50, 75, 100]),
                       yticks = ticks([-50, -30, -10]))

contourf!(ax1, yC, zC, interior(mean(al[:S][end] / al[:VCCC][end], dims=1), 1, jr, kr) .- interior(mean(rl[:S][end] / rl[:VCCC][end], dims=1), 1, jr, kr), levels = -1:0.2:1.4, colormap = :bwr)
contour!(ax1,  yC, zC, interior(mean(al[:b][end] / al[:VCCC][end], dims=1),  1, jr, kr), levels = range(-0.22, -0.17, length=12), color = :black, linestyle=:dash)
contour!(ax1,  yC, zC, interior(mean(rl[:b][end] / rl[:VCCC][end], dims=1),  1, jr, kr), levels = range(-0.22, -0.17, length=12), color = :black)
xlims!(ax1, yC[1], yC[end])
ylims!(ax1, zC[1], zC[end])

ax2  = Axis(fig[1, 2], title = L"\text{RK3-IM - RK3-SE}",
                      ylabel = "",
                      xlabel = L"\text{y [km]}",
                      xticks = ticks([0, 25, 50, 75, 100]),
                      yticks = ([-50, -30, -10], ["", "", ""]))

contourf!(ax2, yC, zC, interior(mean(ri[:S][end] / ri[:VCCC][end], dims=1), 1, jr, kr) .- interior(mean(rl[:S][end] / rl[:VCCC][end], dims=1), 1, jr, kr), levels = -1:0.2:1.4, colormap = :bwr)
contour!(ax2,  yC, zC, interior(mean(ri[:b][end] / ri[:VCCC][end], dims=1),  1, jr, kr), levels = range(-0.22, -0.17, length=12), color = :black, linestyle=:dash)
contour!(ax2,  yC, zC, interior(mean(rl[:b][end] / rl[:VCCC][end], dims=1),  1, jr, kr), levels = range(-0.22, -0.17, length=12), color = :black)
xlims!(ax2, yC[1], yC[end])
ylims!(ax2, zC[1], zC[end])

ax3 = Axis(fig[1, 3], title = L"\text{RK3-SE: \textbf{24 days} - \textbf{40 days}}",
                     ylabel = "",
                     xlabel = L"\text{y [km]}",
                     xticks = ticks([0, 25, 50, 75, 100]),
                     yticks = ([-50, -30, -10], ["", "", ""]))

hm  = contourf!(ax3, yC, zC, interior(mean(rl[:S][290] / rl[:VCCC][290], dims=1), 1, jr, kr) .- interior(mean(rl[:S][end] / rl[:VCCC][end], dims=1), 1, jr, kr), levels = -1:0.2:1.4, colormap = :bwr)
xlims!(ax3, yC[1], yC[end])
ylims!(ax3, zC[1], zC[end])
Colorbar(fig[1, 4], hm, label = L"\Delta \text{S [psu]}", ticks = ticks([-1, -0.5, 0.0, 0.5, 1.0]))
display(fig)
CairoMakie.save(path * "mean-coast.pdf", fig)

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1])
lines!(ax, running_mean(ri[:KE] .- ri[:MKE], 12))
lines!(ax, running_mean(al[:KE] .- al[:MKE], 12))
lines!(ax, running_mean(rl[:KE] .- rl[:MKE], 12))
display(fig)

In [ ]:
fig = Figure(size = (750, 350))
g1  = GridLayout(fig[1, 1]) 
g2  = GridLayout(fig[1, 2]) 
g3  = GridLayout(fig[1, 3]) 
g4  = GridLayout(fig[1, 4]) 
jr  = 1:80
yC  = ri[:S].grid.xᶜᵃᵃ[jr] ./ 1e3
xC  = ri[:S].grid.xᶜᵃᵃ[1:96] ./ 1e3
ax1 = Axis(g1[2, 1], xlabel=L"\text{x [km]}", xticks=ticks([0, 50, 100, 150]), ylabel=L"\text{y [km]}", yticks=ticks([0, 40, 80, 120, 160]))
ax2 = Axis(g2[2, 1], xlabel=L"\text{x [km]}", xticks=ticks([0, 50, 100, 150]), ylabel="", yticks=([0, 40, 80, 120, 160], ["", "", "", "", ""]))
ax3 = Axis(g3[2, 1], xlabel=L"\text{x [km]}", xticks=ticks([0, 50, 100, 150]), ylabel="", yticks=([0, 40, 80, 120, 160], ["", "", "", "", ""]))
ax4 = Axis(g4[2, 1], xlabel=L"\text{x [km]}", xticks=ticks([0, 50, 100, 150]), ylabel="", yticks=([0, 40, 80, 120, 160], ["", "", "", "", ""]))
iter = Observable(1)
un1 = @lift(interior(Field(∂y(rl[:u][$iter] / rl[:VFCC][$iter]) - ∂x(rl[:v][$iter] / rl[:VCFC][$iter])), :, jr, 40) ./ 1e-4)
un2 = @lift(interior(rl[:Abx][$iter], :, jr, 40))
un3 = @lift(interior(rl[:Aby][$iter], :, jr, 40))
un4 = @lift(interior(rl[:Abz][$iter], :, jr, 40))
hm1 = heatmap!(ax1, xC, yC, un1, colorrange=(-1, 1), colormap = :bwr)
hm2 = heatmap!(ax2, xC, yC, un2, colorrange=(-1.5e-3, 1.5e-3), colormap = :bwr)
hm3 = heatmap!(ax3, xC, yC, un3, colorrange=(-1.5e-3, 1.5e-3), colormap = :bwr)
hm4 = heatmap!(ax4, xC, yC, un4, colorrange=(-1.5e-3, 1.5e-3), colormap = :bwr)
Colorbar(g1[1, 1], hm1, label=L"\zeta / f \text{ [-]}", vertical=false, ticks=ticks([-1, -0.5, 0, 0.5, 1]), height=Relative(1/20))
Colorbar(g2[1, 1], hm2, label=L"Px \text{ [m}^2\text{s}^{-5}\text{]}", vertical=false, ticks=([-1e-3, 0, 1e-3], [L"-10^{-3}", L"0", L"10^{-3}"]), height=Relative(1/20))
Colorbar(g3[1, 1], hm3, label=L"Py \text{ [m}^2\text{s}^{-5}\text{]}", vertical=false, ticks=([-1e-3, 0, 1e-3], [L"-10^{-3}", L"0", L"10^{-3}"]), height=Relative(1/20))
Colorbar(g4[1, 1], hm4, label=L"Pz \text{ [m}^2\text{s}^{-5}\text{]}", vertical=false, ticks=([-1e-3, 0, 1e-3], [L"-10^{-3}", L"0", L"10^{-3}"]), height=Relative(1/20))
rowgap!(g1, -50)
rowgap!(g2, -50)
rowgap!(g3, -50)
rowgap!(g4, -50)

iter[] = 481
display(current_figure())

CairoMakie.save(path * "snapshots-coast.pdf", fig)

In [ ]:
fig = Figure(size = (750, 350))
g1  = GridLayout(fig[1, 1]) 
g2  = GridLayout(fig[1, 2]) 
g3  = GridLayout(fig[1, 3]) 
g4  = GridLayout(fig[1, 4]) 
jr  = 1:80
yC  = ri[:S].grid.xᶜᵃᵃ[jr] ./ 1e3
xC  = ri[:S].grid.xᶜᵃᵃ[1:96] ./ 1e3
ax1 = Axis(g1[2, 1], xlabel=L"\text{x [km]}", xticks=ticks([0, 50, 100, 150]), ylabel=L"\text{y [km]}", yticks=ticks([0, 40, 80, 120, 160]))
ax2 = Axis(g2[2, 1], xlabel=L"\text{x [km]}", xticks=ticks([0, 50, 100, 150]), ylabel="", yticks=([0, 40, 80, 120, 160], ["", "", "", "", ""]))
ax3 = Axis(g3[2, 1], xlabel=L"\text{x [km]}", xticks=ticks([0, 50, 100, 150]), ylabel="", yticks=([0, 40, 80, 120, 160], ["", "", "", "", ""]))
ax4 = Axis(g4[2, 1], xlabel=L"\text{x [km]}", xticks=ticks([0, 50, 100, 150]), ylabel="", yticks=([0, 40, 80, 120, 160], ["", "", "", "", ""]))
iter = Observable(1)
un1 = @lift(interior(Field(∂y(ri[:u][$iter] / ri[:VFCC][$iter]) - ∂x(ri[:v][$iter] / ri[:VCFC][$iter])), :, jr, 40) ./ 1e-4)
un2 = @lift(interior(ri[:Abx][$iter], :, jr, 40))
un3 = @lift(interior(ri[:Aby][$iter], :, jr, 40))
un4 = @lift(interior(ri[:Abz][$iter], :, jr, 40))
hm1 = heatmap!(ax1, xC, yC, un1, colorrange=(-1, 1), colormap = :bwr)
hm2 = heatmap!(ax2, xC, yC, un2, colorrange=(-1.5e-3, 1.5e-3), colormap = :bwr)
hm3 = heatmap!(ax3, xC, yC, un3, colorrange=(-1.5e-3, 1.5e-3), colormap = :bwr)
hm4 = heatmap!(ax4, xC, yC, un4, colorrange=(-1.5e-3, 1.5e-3), colormap = :bwr)
Colorbar(g1[1, 1], hm1, label=L"\zeta / f \text{ [-]}", vertical=false, ticks=ticks([-1, -0.5, 0, 0.5, 1]), height=Relative(1/20))
Colorbar(g2[1, 1], hm2, label=L"Px \text{ [m}^2\text{s}^{-5}\text{]}", vertical=false, ticks=([-1e-3, 0, 1e-3], [L"-10^{-3}", L"0", L"10^{-3}"]), height=Relative(1/20))
Colorbar(g3[1, 1], hm3, label=L"Py \text{ [m}^2\text{s}^{-5}\text{]}", vertical=false, ticks=([-1e-3, 0, 1e-3], [L"-10^{-3}", L"0", L"10^{-3}"]), height=Relative(1/20))
Colorbar(g4[1, 1], hm4, label=L"Pz \text{ [m}^2\text{s}^{-5}\text{]}", vertical=false, ticks=([-1e-3, 0, 1e-3], [L"-10^{-3}", L"0", L"10^{-3}"]), height=Relative(1/20))
rowgap!(g1, -50)
rowgap!(g2, -50)
rowgap!(g3, -50)
rowgap!(g4, -50)

iter[] = 481
display(current_figure())


In [54]:
record(fig, "test2.mp4", 1:481) do i
   @info "step $i";
   iter[] = i;
end

[ Info: step 1
[ Info: step 2
[ Info: step 3
[ Info: step 4
[ Info: step 5
[ Info: step 6
[ Info: step 7
[ Info: step 8
[ Info: step 9
[ Info: step 10
[ Info: step 11
[ Info: step 12
[ Info: step 13
[ Info: step 14
[ Info: step 15
[ Info: step 16
[ Info: step 17
[ Info: step 18
[ Info: step 19
[ Info: step 20
[ Info: step 21
[ Info: step 22
[ Info: step 23
[ Info: step 24
[ Info: step 25
[ Info: step 26
[ Info: step 27
[ Info: step 28
[ Info: step 29
[ Info: step 30
[ Info: step 31
[ Info: step 32
[ Info: step 33
[ Info: step 34
[ Info: step 35
[ Info: step 36
[ Info: step 37
[ Info: step 38
[ Info: step 39
[ Info: step 40
[ Info: step 41
[ Info: step 42
[ Info: step 43
[ Info: step 44
[ Info: step 45
[ Info: step 46
[ Info: step 47
[ Info: step 48
[ Info: step 49
[ Info: step 50
[ Info: step 51
[ Info: step 52
[ Info: step 53
[ Info: step 54
[ Info: step 55
[ Info: step 56
[ Info: step 57
[ Info: step 58
[ Info: step 59
[ Info: step 60
[ Info: step 61
[ Info: step 62
[ Info: step 63
[

"test2.mp4"

In [ ]:
fig = Figure(size = (750, 250))
times = ri[:Abx].times ./ 86400
ax = Axis(fig[1, 1], xlabel=L"\text{time [days]}", xticks=ticks([0, 10, 20, 30, 40]), yticks = ([0, 2e-7, 4e-7, 6e-7], [L"0", L"2\cdot10^{-7}", L"4\cdot10^{-7}", L"6\cdot10^{-7}"]), title=L"|\nabla b|^2 \text{ [s}^{-4}\text{]}")
lines!(ax, times, rl[:gbt], color=c1)
lines!(ax, times, ri[:gbt], color=c2)
lines!(ax, times, al[:gbt], color=c3)
xlims!(ax, 0, 40)

ax = Axis(fig[1, 2], xlabel=L"\text{time [days]}", xticks=ticks([0, 10, 20, 30, 40]), yticks=ticks([0, 10, 20, 30]), title=L"\text{TKE [cm}^2\text{s}^{-2}\text{]}")
lines!(ax, times, (rl[:KE] .- rl[:MKE]) .* 1e4, color=c1)
lines!(ax, times, (ri[:KE] .- ri[:MKE]) .* 1e4, color=c2)
lines!(ax, times, (al[:KE] .- al[:MKE]) .* 1e4, color=c3)
xlims!(ax, 0, 40)

ax = Axis(fig[1, 3], xlabel=L"\text{time [days]}", xticks=ticks([0, 10, 20, 30, 40]), yticks=ticks([5, 10, 15, 20]), title=L"\eta^2 \text{ [cm}^2\text{]}")
lines!(ax, times, rl[:η2] .* 1e4, color=(c1, 0.15))
lines!(ax, times, ri[:η2] .* 1e4, color=(c2, 0.15))
lines!(ax, times, al[:η2] .* 1e4, color=(c3, 0.15))
lines!(ax, running_mean(times, 12), running_mean(rl[:η2], 12) .* 1e4, color=c1)
lines!(ax, running_mean(times, 12), running_mean(ri[:η2], 12) .* 1e4, color=c2)
lines!(ax, running_mean(times, 12), running_mean(al[:η2], 12) .* 1e4, color=c3)
xlims!(ax, 0, 40)
ylims!(ax, 5, 20)

display(fig)
CairoMakie.save(path * "energetics-coast.pdf", fig)

In [ ]:
# Compute the spectra
Nx, Ny, Nz = size(rl[:u][end])
Url, freq = TimestepperTestCases.y_average_spectra(Field(rl[:u][end] / rl[:VFCC][end]), 1:Nx, 1:Ny÷2; k=Nz)
Vrl, _    = TimestepperTestCases.y_average_spectra(Field(rl[:v][end] / rl[:VCFC][end]), 1:Nx, 1:Ny÷2; k=Nz)
Brl, _    = TimestepperTestCases.y_average_spectra(Field(rl[:b][end] / rl[:VCCC][end]), 1:Nx, 1:Ny÷2; k=Nz)

Uri, _ = TimestepperTestCases.y_average_spectra(Field(ri[:u][end] / rl[:VFCC][end]), 1:Nx, 1:Ny÷2; k=Nz)
Vri, _ = TimestepperTestCases.y_average_spectra(Field(ri[:v][end] / rl[:VCFC][end]), 1:Nx, 1:Ny÷2; k=Nz)
Bri, _ = TimestepperTestCases.y_average_spectra(Field(ri[:b][end] / ri[:VCCC][end]), 1:Nx, 1:Ny÷2; k=Nz)

Ual, _ = TimestepperTestCases.y_average_spectra(Field(al[:u][end] / rl[:VFCC][end]), 1:Nx, 1:Ny÷2; k=Nz)
Val, _ = TimestepperTestCases.y_average_spectra(Field(al[:v][end] / rl[:VCFC][end]), 1:Nx, 1:Ny÷2; k=Nz)
Bal, _ = TimestepperTestCases.y_average_spectra(Field(al[:b][end] / al[:VCCC][end]), 1:Nx, 1:Ny÷2; k=Nz)

specUrl = real.(Url) ./ 81
specVrl = real.(Vrl) ./ 81
specBrl = real.(Brl) ./ 81
specUri = real.(Uri) ./ 81
specVri = real.(Vri) ./ 81
specBri = real.(Bri) ./ 81
specUal = real.(Ual) ./ 81
specVal = real.(Val) ./ 81
specBal = real.(Bal) ./ 81

for t in 400:480
    Url, freq = TimestepperTestCases.y_average_spectra(Field(rl[:u][t] / rl[:VFCC][t]), 1:Nx, 1:Ny÷2; k=Nz)
    Vrl, _    = TimestepperTestCases.y_average_spectra(Field(rl[:v][t] / rl[:VCFC][t]), 1:Nx, 1:Ny÷2; k=Nz)
    Brl, _    = TimestepperTestCases.y_average_spectra(Field(rl[:b][t] / rl[:VCCC][t]), 1:Nx, 1:Ny÷2; k=Nz)
    
    Uri, _ = TimestepperTestCases.y_average_spectra(Field(ri[:u][t] / ri[:VFCC][t]), 1:Nx, 1:Ny÷2; k=Nz)
    Vri, _ = TimestepperTestCases.y_average_spectra(Field(ri[:v][t] / ri[:VCFC][t]), 1:Nx, 1:Ny÷2; k=Nz)
    Bri, _ = TimestepperTestCases.y_average_spectra(Field(ri[:b][t] / ri[:VCCC][t]), 1:Nx, 1:Ny÷2; k=Nz)

    Ual, _ = TimestepperTestCases.y_average_spectra(Field(al[:u][t] / al[:VFCC][t]), 1:Nx, 1:Ny÷2; k=Nz)
    Val, _ = TimestepperTestCases.y_average_spectra(Field(al[:v][t] / al[:VCFC][t]), 1:Nx, 1:Ny÷2; k=Nz)
    Bri, _ = TimestepperTestCases.y_average_spectra(Field(al[:b][t] / al[:VCCC][t]), 1:Nx, 1:Ny÷2; k=Nz)

    
    specUrl .+= real.(Url) ./ 81
    specVrl .+= real.(Vrl) ./ 81
    specBrl .+= real.(Brl) ./ 81
    specUri .+= real.(Uri) ./ 81
    specVri .+= real.(Vri) ./ 81
    specBri .+= real.(Bri) ./ 81
    specUal .+= real.(Ual) ./ 81
    specVal .+= real.(Val) ./ 81
    specBal .+= real.(Bal) ./ 81
end


In [ ]:
fig = Figure()
ax  = Axis(fig[1, 1], yscale=log10, xscale=log10)
lines!(ax, freq[2:end], specBrl[2:end]) # .+ specVrl[2:end])
lines!(ax, freq[2:end], specBri[2:end]) # .+ specVri[2:end])
lines!(ax, freq[2:end], specBal[2:end]) # .+ specVal[2:end])
display(fig)